In [1]:
import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def hello_world(name):
    return f"hello .... {name} .. Hi!!"
hello_world("Ammar")


'hello .... Ammar .. Hi!!'

In [ ]:
interface = gr.Interface(fn = hello_world, inputs="text", outputs = "text")
interface.launch()

### Customised Components in Gradio

In [ ]:
interface = gr.Interface(fn = hello_world, inputs = gr.inputs.Textbox(lines=5, placeholder="Enter your input here ... "), outputs = "text")
interface.launch()

#### Sepia filter example (image input)

In [ ]:
import numpy as np

def sepia(input_img):
    sepia_filter = np.array([[.393, .769, .189], [.349, .686, .168], [.272, .534, .131]])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

interface = gr.Interface(fn = sepia, inputs = gr.inputs.Image(shape=(200, 200)), outputs = "image")
interface.launch()

### Tabular data as input

In [ ]:


def sales_projections(employee_data):
    sales_data = employee_data.iloc[:, 1:4].astype("int").to_numpy()
    regression_values = np.apply_along_axis(lambda row: 
        np.array(np.poly1d(np.polyfit([0,1,2], row, 2))), 0, sales_data)
    projected_months = np.repeat(np.expand_dims(
        np.arange(3,12), 0), len(sales_data), axis=0)
    projected_values = np.array([
        month * month * regression[0] + month * regression[1] + regression[2]
        for month, regression in zip(projected_months, regression_values)])
    plt.plot(projected_values.T)
    plt.legend(employee_data["Name"])
    return employee_data, plt.gcf(), regression_values

iface = gr.Interface(sales_projections, 
    gr.inputs.Dataframe(
        headers=["Name", "Jan Sales", "Feb Sales", "Mar Sales"],
        default=[["Jon", 12, 14, 18], ["Alice", 14, 17, 2], ["Sana", 8, 9.5, 12]]
    ),
    [
        "dataframe",
        "plot",
        "numpy"
    ],
    description="Enter sales figures for employees to predict sales trajectory over year."
)
iface.launch()

#### **How to use multiple inputs in gradio applications**

In [16]:
def BMI(name, height, weight, feeling):
    bmiVal = weight / (height**2)
    result_emoticon = "😊" if bmiVal < 10 else "😢"
    output_str = f"Hello 😉 {name} ... \nyour BMI is.... {bmiVal}"
    txt = "Happy" if feeling else "Sad"
    return (output_str, result_emoticon, txt)

# BMI("ammar", 1.2, 80, True)

In [ ]:
interface = gr.Interface(
    fn = BMI,
    inputs = ['text', gr.inputs.Slider(0, 200, label= 'Height in Meters'), gr.inputs.Slider(0, 100, label="Weight in Kg"), gr.inputs.Checkbox(label="feeling happy")],
    outputs = ['text', 'text', 'text'],
    examples = [['ammar', 175,50, True], 
                ['Someone', 180, 65, True], 
                ['No-one', 190, 99, False]],
    description="Flag if you find any erroneous output",
    # theme="darkhuggingface",
    css = """
    body {
        background-color: grey;
    }
    """
)
interface.launch(auth={'ammar', 'Something@123'}, auth_message = "Check your <strong>login details</strong> sent to your <i>email</i>")

In [1]:
import gradio as gr
from transformers import pipeline
sentiment = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100%|██████████| 256M/256M [00:17<00:00, 15.6MB/s] 
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Downloading: 100%|██████████| 48.0/48.0 [00:00<?, ?B/s]
Downloading: 100%|██████████| 226k/226k [00:02<00:00, 109kB/s]  


In [3]:
def get_sentiment(input_text):
    return sentiment(input_text)

result = get_sentiment("The movie was very bad")
print(result)

[{'label': 'NEGATIVE', 'score': 0.9997990727424622}]


In [5]:
iface = gr.Interface(fn = get_sentiment, inputs = "text", outputs = ["text"], title="Sentiment Analysis", description="Get Sentiment Negative / Positive for the given input" )
iface.launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x1eaa50272b0>,
 'http://127.0.0.1:7860/',
 None)